# Imports

In [1]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")



# Load datasets

In [2]:
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')

train_targets_a = pd.read_parquet('../data/A/train_targets.parquet')
train_targets_b = pd.read_parquet('../data/B/train_targets.parquet')
train_targets_c = pd.read_parquet('../data/C/train_targets.parquet')


# Data clean up

In [3]:

# Data set A, B and C clean up

def data_clean_up(x_train_est, x_train_observe, y_train):

  if 'date_calc' in x_train_est.columns:
    x_train_est.drop(columns="date_calc", inplace=True)

  x_train = pd.concat([x_train_observe, x_train_est])

  # Group the rows into blocks of 4 and apply the aggregation function
  agg_func = {col: 'mean' for col in x_train.columns[1:]}
  X_train_downscaled = x_train.groupby(x_train.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})

  y_train.dropna(inplace=True)
  combined_data = pd.merge(X_train_downscaled, y_train, left_on='date_forecast', right_on='time')
  y_train = combined_data['pv_measurement']

  if 'date_forecast' and 'time' and 'pv_measurement' in combined_data.columns:
    #combined_data.drop(columns="date_forecast", inplace=True)
    combined_data.drop(columns=['time', 'pv_measurement'], inplace=True)
    combined_data = combined_data(columns=[ 'direct_rad:W', 'clear_sky_rad:W', 'date_forecast'])


  return combined_data, y_train

x_train_a, y_train_a = data_clean_up(X_train_estimated_a, X_train_observed_a, train_targets_a)
x_train_b, y_train_b = data_clean_up(X_train_estimated_b, X_train_observed_b, train_targets_b)
x_train_c, y_train_c = data_clean_up(X_train_estimated_c, X_train_observed_c, train_targets_c)


def data_clean_up_test(x_test_est):


  # Group the rows into blocks of 4 and apply the aggregation function
  agg_func = {col: 'mean' for col in x_test_est.columns[1:]}
  X_test_downscaled = x_test_est.groupby(x_test_est.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})

  #if 'date_forecast' in X_test_downscaled.columns:
    #X_test_downscaled.drop(columns="date_forecast", inplace=True)

  return X_test_downscaled

X_test_estimated_a = data_clean_up_test(X_test_estimated_a)
X_test_estimated_b = data_clean_up_test(X_test_estimated_b)
X_test_estimated_c = data_clean_up_test(X_test_estimated_c)


# Feature engineering

In [4]:
# Do feature selection etc.

# Polynomial features of degree 2 of most important features

def polynomial_feature(x_dataset, features):
  
  for feature in features:
    x_dataset[feature + ':squared'] = x_dataset[feature] ** 2
    x_dataset[feature + ':cubed'] = x_dataset[feature] ** 3
    x_dataset[feature + ':sqrt'] = np.sqrt(x_dataset[feature])

  return x_dataset


# x_train_a = polynomial_feature(x_train_a, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])
# x_train_b = polynomial_feature(x_train_b, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])
# x_train_c = polynomial_feature(x_train_c, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])

# X_test_estimated_a = polynomial_feature(X_test_estimated_a, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])
# X_test_estimated_b = polynomial_feature(X_test_estimated_b, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])
# X_test_estimated_c = polynomial_feature(X_test_estimated_c, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])

def drop_features(x_dataset, features):
  for feature in features:
    x_dataset.drop(columns=feature, inplace=True)

  return x_dataset

# x_train_a = drop_features(x_train_a, ['snow_density:kgm3', 'fresh_snow_3h:cm'])
# x_train_b = drop_features(x_train_b, ['snow_density:kgm3', 'fresh_snow_3h:cm'])
# x_train_c = drop_features(x_train_c, ['snow_density:kgm3', 'fresh_snow_3h:cm'])

# X_test_estimated_a = drop_features(X_test_estimated_a, ['snow_density:kgm3', 'fresh_snow_3h:cm'])
# X_test_estimated_b = drop_features(X_test_estimated_b, ['snow_density:kgm3', 'fresh_snow_3h:cm'])
# X_test_estimated_c = drop_features(X_test_estimated_c, ['snow_density:kgm3', 'fresh_snow_3h:cm'])



In [5]:
def get_unixtime(datetime: pd.Series) -> pd.Series:
    unixtime = (datetime - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    return unixtime

xat_datetime = x_train_a['date_forecast']
xbt_datetime = x_train_b['date_forecast']
xct_datetime = x_train_c['date_forecast']

xat_unixtime = get_unixtime(xat_datetime)
xbt_unixtime = get_unixtime(xbt_datetime)
xct_unixtime = get_unixtime(xct_datetime)

XTA_unix = get_unixtime(X_test_estimated_a['date_forecast'])
XTB_unix = get_unixtime(X_test_estimated_b['date_forecast'])
XTC_unix = get_unixtime(X_test_estimated_c['date_forecast'])

## We now need functions for assigning daily and yearly cycles (described in datanalysis docu on Peter branch)
# plus 2 avoids 0 and negative values
day = 24*60*60
year = (365.2425)*day

def sinus_day(unix_time):
    return 2 + np.sin(unix_time * (2 * np.pi / day)) # since it is seconds since 1.1.1970 we divide by seconds in a day to get seasonal changes throughout the dat

def sinus_year(unix_time):
    return 2+ np.sin(unix_time * (2 * np.pi / year))

def cosinus_day(unix_time):
    return 2+np.cos(unix_time * (2 * np.pi / day))

def cosinus_year(unix_time):
    return 2+np.cos(unix_time * (2 * np.pi / year))

# function for returning two series with the daily cycles (sine and cosine)
def get_daycycle(unixtime: pd.Series) -> (pd.Series, pd.Series):
    sinus_daytime = unixtime.apply(sinus_day)
    sinus_daytime = sinus_daytime.rename('sinus_day') 
    cosinus_daytime = unixtime.apply(cosinus_day)
    cosinus_daytime = cosinus_daytime.rename('cosine_day')
    return sinus_daytime, cosinus_daytime

# Function for returning two series with the yearly cycles
def get_yearcycle(unixtime: pd.Series) -> (pd.Series, pd.Series):
    sinus_yeartime = unixtime.apply(sinus_year)
    sinus_yeartime = sinus_yeartime.rename('sinus_year')
    cosinus_yeartime = unixtime.apply(cosinus_year)
    cosinus_yeartime = cosinus_yeartime.rename('cosinus_year')
    return sinus_yeartime, cosinus_yeartime

xat_day_sin, xat_day_cos = get_daycycle(xat_unixtime)
xat_year_sin, xat_year_cos = get_yearcycle(xat_unixtime)
xbt_day_sin, xbt_day_cos = get_daycycle(xbt_unixtime)
xbt_year_sin, xbt_year_cos = get_yearcycle(xbt_unixtime)
xct_day_sin, xct_day_cos = get_daycycle(xct_unixtime)
xct_year_sin, xct_year_cos = get_yearcycle(xct_unixtime)

XTA_day_sin, XTA_day_cos = get_daycycle(XTA_unix)
XTA_year_sin, XTA_year_cos = get_yearcycle(XTA_unix)
XTB_day_sin, XTB_day_cos = get_daycycle(XTB_unix)
XTB_year_sin, XTB_year_cos = get_yearcycle(XTB_unix)
XTC_day_sin, XTC_day_cos = get_daycycle(XTC_unix)
XTC_year_sin, XTC_year_cos = get_yearcycle(XTC_unix)

x_train_a = x_train_a.join([xat_day_sin, xat_day_cos, xat_year_sin, xat_year_cos])
x_train_b = x_train_b.join([xbt_day_sin, xbt_day_cos, xbt_year_sin, xbt_year_cos])
x_train_c = x_train_c.join([xct_day_sin, xct_day_cos, xct_year_sin, xct_year_cos])
x_train_a.drop(columns=['date_forecast'], inplace=True)
x_train_b.drop(columns=['date_forecast'], inplace=True)
x_train_c.drop(columns=['date_forecast'], inplace=True)

X_test_estimated_a = X_test_estimated_a.join([XTA_day_sin, XTA_day_cos, XTA_year_sin, XTA_year_cos])
X_test_estimated_b = X_test_estimated_b.join([XTB_day_sin, XTB_day_cos, XTB_year_sin, XTB_year_cos])
X_test_estimated_c = X_test_estimated_c.join([XTC_day_sin, XTC_day_cos, XTC_year_sin, XTC_year_cos])
X_test_estimated_a.drop(columns=['date_forecast'], inplace=True)
X_test_estimated_b.drop(columns=['date_forecast'], inplace=True)
X_test_estimated_c.drop(columns=['date_forecast'], inplace=True)



The different loations has different priorities on the dataset which might mean that we can drop different cells to get better results per location. 
```python
data.drop(columns=['elevation:m', 
                                #'is_day:idx',
                                'elevation'
                                'wind_speed_u_10m:ms',
                                'wind_speed_v_10m:ms',
                                'sfc_pressure:hPa',
                                'pressure_100m:hPa',
                                'pressure_50m:hPa',
                                'msl_pressure:hPa',
                                #'diffuse_rad_1h:J',
                                #'direct_rad_1h:J',
                                'air_density_2m:kgm3'], inplace=True)
```

# Training the model

In [6]:

# Split the data into training and validation sets

x_train_a, x_val_a, y_train_a, y_val_a = train_test_split(x_train_a, y_train_a, test_size=0.17, random_state=42)
x_train_b, x_val_b, y_train_b, y_val_b = train_test_split(x_train_b, y_train_b, test_size=0.17, random_state=42)
x_train_c, x_val_c, y_train_c, y_val_c = train_test_split(x_train_c, y_train_c, test_size=0.17, random_state=42)

model_a = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.25, max_depth=10, min_child_weight=2, gamma=150, reg_lambda=20)
model_b = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.25, max_depth=10, min_child_weight=4, gamma=34, reg_lambda=20)
model_c = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.24, max_depth=10, min_child_weight=3, gamma=8, reg_lambda=44)
# max_depth = 6 gives best


model_a.fit(x_train_a, y_train_a)
model_b.fit(x_train_b, y_train_b)
model_c.fit(x_train_c, y_train_c)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=8, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.24, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

# Evaluate prediction

In [7]:
# Evaluate the model based on the validation data

mse_a = mean_squared_error(y_val_a, model_a.predict(x_val_a))
print("MSE for A: ", mse_a)
mse_b = mean_squared_error(y_val_b, model_b.predict(x_val_b))
print("MSE for B: ", mse_b)
mse_c = mean_squared_error(y_val_c, model_c.predict(x_val_c))
print("MSE for C: ", mse_c)
print("Mean MSE: ", (mse_a + mse_b + mse_c) / 3)

# Evaluate the predictions

score_a = model_a.score(x_val_a, y_val_a)
score_b = model_b.score(x_val_b, y_val_b)
score_c = model_c.score(x_val_c, y_val_c)

print("Score A: ", score_a)
print("Score B: ", score_b)
print("Score C: ", score_c)
print('')

# Get feature importance scores
models = [(model_a, 'A'), (model_b, 'B'), (model_c, 'C')]
for model in models:

    feature_importance_scores = model[0].feature_importances_

# Create a DataFrame to associate features with their importance scores
    feature_importance_df1 = pd.DataFrame({'Feature': x_train_a.columns, 'Importance': feature_importance_scores})

# Sort features by importance in descending order
    feature_importance_df1 = feature_importance_df1.sort_values(by='Importance', ascending=False)

# Print or visualize the feature importance scores
    
    print(f'Model {model[1]}')
    print(feature_importance_df1.head(10))
    print('')


MSE for A:  150838.40006252527
MSE for B:  4013.961205795114
MSE for C:  2366.6758964492474
Mean MSE:  52406.345721589874
Score A:  0.8902033621067098
Score B:  0.8958001762415478
Score C:  0.9206957504873496

Model A
                 Feature  Importance
8           direct_rad:W    0.545097
6          diffuse_rad:W    0.073984
45          cosinus_year    0.025347
18      is_in_shadow:idx    0.025004
30    snow_melt_10min:mm    0.023213
3        clear_sky_rad:W    0.022969
32         sun_azimuth:d    0.021597
21  precip_type_5min:idx    0.016019
20        precip_5min:mm    0.015906
4        dew_or_rime:idx    0.013984

Model B
              Feature  Importance
8        direct_rad:W    0.339489
33    sun_elevation:d    0.150946
3     clear_sky_rad:W    0.096186
18   is_in_shadow:idx    0.087651
17         is_day:idx    0.028097
25    rain_water:kgm2    0.027256
45       cosinus_year    0.024585
14  fresh_snow_24h:cm    0.022144
44         sinus_year    0.015874
32      sun_azimuth:d    0

```
Most_common = ['direct_rad:W', 'clear_sky_rad:W']

MSE for A:  155326.11984010294
MSE for B:  4311.822664627681
MSE for C:  2484.332046556924
Mean MSE:  54040.75818376252
Score A:  0.8869367102250868
Score B:  0.8880678863853381
Score C:  0.9167532450100108

Model A
                 Feature  Importance
9           direct_rad:W    0.558548 <-------- 3
7          diffuse_rad:W    0.077622 <-------- 2
18      is_in_shadow:idx    0.028293 <--------2
3        clear_sky_rad:W    0.026308 <-------- 3
40          cosinus_year    0.024731
24     snow_density:kgm3    0.022149 <-------- 2
29         sun_azimuth:d    0.021677
20  precip_type_5min:idx    0.016280
6         dew_point_2m:K    0.015750
19        precip_5min:mm    0.014176 <-------- 2

Model B
             Feature  Importance
9       direct_rad:W    0.369634     <--------- 3
30   sun_elevation:d    0.165624     <-------- 2
3    clear_sky_rad:W    0.084479     <--------- 3
18  is_in_shadow:idx    0.074470     <---------
17        is_day:idx    0.037519
22   rain_water:kgm2    0.028054
40      cosinus_year    0.024928
7      diffuse_rad:W    0.019828     <--------- 2
39        sinus_year    0.016917
16  fresh_snow_6h:cm    0.015372     <-------- 1/2

Model C
                 Feature  Importance
30       sun_elevation:d    0.737467 <--------- 2
3        clear_sky_rad:W    0.110762 <--------- 3
9           direct_rad:W    0.029844 <--------- 3
10       direct_rad_1h:J    0.023283
20  precip_type_5min:idx    0.010948
12     fresh_snow_12h:cm    0.010607
24     snow_density:kgm3    0.009615 <-------- 2
14     fresh_snow_24h:cm    0.007877 <-------- 1/2
6         dew_point_2m:K    0.005590
19        precip_5min:mm    0.005087 <-------- 2
```

# Make predictions

In [10]:
model_a = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.25, max_depth=10, min_child_weight=2, gamma=150, reg_lambda=20)
model_b = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.25, max_depth=10, min_child_weight=4, gamma=34, reg_lambda=20)
model_c = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.24, max_depth=10, min_child_weight=3, gamma=8, reg_lambda=44)

# max_depth = 6 gives best
model_a.fit(x_train_a, y_train_a)
model_b.fit(x_train_b, y_train_b)
model_c.fit(x_train_c, y_train_c)

y_pred_a = model_a.predict(X_test_estimated_a)
y_pred_b = model_b.predict(X_test_estimated_b)
y_pred_c = model_c.predict(X_test_estimated_c)

y_pred = np.concatenate((y_pred_a, y_pred_b, y_pred_c), axis=0)



In [12]:
for i in range(len(y_pred)):
    if y_pred[i] < 0: 
        y_pred[i] = 0



array([  0.     ,   0.     ,   0.     ,  94.73076, 467.24115],
      dtype=float32)

# Create submission

In [13]:
y_test_pred = y_pred

test = pd.read_csv('../data/test.csv')
test['prediction'] = y_test_pred
sample_submission = pd.read_csv('../data/sample_submission.csv')
submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
submission.to_csv('submission.csv', index=False)